In [17]:
import pandas as pd

In [18]:
import io
with io.open('./Readme.txt', 'r', encoding='utf-8') as f:
    print(f.read())

-- Описание задания

Мы предлагаем Вам попробовать свои силы в решении небольшой задачки. Как известно, блокчейн биткоина хранит все когда-либо используемые адреса и транзакции между ними. Несмотря на кажущуюся анонимность данной криптовалюты, есть широкоизвестные методы, которые позволяют связать множество адресов, предполагая что они принадлежат одному и тому же владельцу (это может быть человек или компания). Так как все транзакции между адресами прозрачны, мы можем видеть сколько денег ушло от одного владельца к другому (это называется сash flow). Мы предоставляем такую услугу нашим клиентам и предлагаем Вам посчитать сash flow между двумя кластерами адресов.

-- Необходимые предметные знания

Транзакция биткоина представляет собой набор входных и выходных адресов, а также указание сколько каждый адрес вносит биткоинов в данную транзакцию (для входных) или забирает из транзакции (для выходных адресов). Стоит заметить, что ничего не известно о том, сколько биткоинов переправляет кон

In [19]:
df_clust = pd.read_csv('./address_clust.csv')
df_clust.head()

,address_id,cluster_id
0,71413451,1
1,71411914,1
2,71410369,1
3,71410100,1
4,71410040,1


In [20]:
df_stats = pd.read_csv('./address_stats.csv')
df_stats.head()

,id,address_id,transaction_id,received,sent
0,87134765,46402336,19162323,0.0,1800000.0
1,87134766,45919235,19162323,0.0,1071052.0
2,87134767,46529090,19162323,1000000.0,0.0
3,87134768,46529091,19162323,1821052.0,0.0
4,87154706,46529090,19166856,0.0,1000000.0


Объединение таблиц по значению address_id, задание адресам, не входящих в address_clust значение кластера "0"

In [21]:
df = pd.merge(df_stats, df_clust, how='left', on='address_id')
df['cluster_id'].fillna(value=0, inplace=True);

In [22]:
df.sort_values(by=['transaction_id']).head()

,id,address_id,transaction_id,received,sent,cluster_id
0,87134765,46402336,19162323,0.0,1800000.0,0.0
1,87134766,45919235,19162323,0.0,1071052.0,0.0
2,87134767,46529090,19162323,1000000.0,0.0,1.0
3,87134768,46529091,19162323,1821052.0,0.0,0.0
4,87154706,46529090,19166856,0.0,1000000.0,1.0


Проверка на пустые значения:

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 76724 entries, 0 to 76723
Data columns (total 6 columns):
id                76724 non-null int64
address_id        76724 non-null int64
transaction_id    76724 non-null int64
received          76724 non-null float64
sent              76724 non-null float64
cluster_id        76724 non-null float64
dtypes: float64(3), int64(3)
memory usage: 4.1 MB


Группировка по transaction_id - каждая группа это отдельная транзакция

In [24]:
df_grouped = df.groupby('transaction_id')

Посмотрим, какое количество кластеров может учавствовать в транзакции:

In [25]:
uniq_1 = 0
uniq_several = 0
for name, group in df_grouped:
    x = group[group['sent'] > 0]['cluster_id'].nunique()
    if x == 1: uniq_1 +=1
    if x > 1: uniq_several +=1

print('Количество транзакций, в которых биткоины отправляются с адресов одного кластера - {}'.format(uniq_1))
print('Количество транзакций, в которых биткоины отправляются с адресов нескольких кластеров - {}'.format(uniq_several))

Количество транзакций, в которых биткоины отправляются с адресов одного кластера - 13753
Количество транзакций, в которых биткоины отправляются с адресов нескольких кластеров - 0


In [26]:
uniq_1 = 0
uniq_several = 0
for name, group in df_grouped:
    x = group[group['received'] > 0]['cluster_id'].nunique()
    if x == 1: uniq_1 +=1
    if x > 1: uniq_several +=1

print('Количество транзакций, в которых биткоины приходят на адреса одного кластера - {}'.format(uniq_1))
print('Количество транзакций, в которых биткоины приходят на адреса нескольких кластеров - {}'.format(uniq_several))

Количество транзакций, в которых биткоины приходят на адреса одного кластера - 2337
Количество транзакций, в которых биткоины приходят на адреса нескольких кластеров - 11416


Так как в транзакции в отправке биткоинов учавствует только один кластер, необходимо отобрать те группы (транзакции), где отправляет биткоины интересующий нас кластер. Если взять из этой группы количество биткоинов, полученных интересующей группой, получится ответ - сколько биткоинов было отправлено с одного кластера на другой.

In [27]:
def get_flow(df, sent_name, received_name):
    sent = 0
    sent_name = int(sent_name)
    received_name = int(received_name)
    for name, group in df:
        if group[group['cluster_id'] == sent_name]['sent'].sum() == 0: continue
        sent += group[group['cluster_id'] == received_name]['received'].sum()
    return sent

In [28]:
def get_fee(df, sent_name):
    fee = 0
    sent_name = int(sent_name)
    for name, group in df:
        if group[group['cluster_id'] == sent_name]['sent'].sum() == 0: continue
        fee += group['sent'].sum() - group['received'].sum()
    return fee

In [29]:
def satoshi_to_btc(satoshi):
    return satoshi / 1e8

In [30]:
for i in range(3):
    for j in range(3):
        sent = get_flow(df_grouped, i, j)
        sent = satoshi_to_btc(sent)
        print('Отправлено с кластера {} на кластер {} {:.2f} BTC'.format(i, j, sent))

Отправлено с кластера 0 на кластер 0 722.75 BTC
Отправлено с кластера 0 на кластер 1 297.12 BTC
Отправлено с кластера 0 на кластер 2 9.74 BTC
Отправлено с кластера 1 на кластер 0 300.55 BTC
Отправлено с кластера 1 на кластер 1 209.80 BTC
Отправлено с кластера 1 на кластер 2 129.24 BTC
Отправлено с кластера 2 на кластер 0 4.87 BTC
Отправлено с кластера 2 на кластер 1 135.82 BTC
Отправлено с кластера 2 на кластер 2 889.81 BTC


In [31]:
for i in range(0,3):
    fee = get_fee(df_grouped, i)
    fee = satoshi_to_btc(fee)
    print('Кластер {} потратил на fee {:.6f} BTC'.format(i, fee))
    

Кластер 0 потратил на fee 0.951604 BTC
Кластер 1 потратил на fee 0.667144 BTC
Кластер 2 потратил на fee 0.745667 BTC
